In [1]:
from datasets import load_dataset


In [9]:
ds = load_dataset("Helsinki-NLP/un_pc", "ar-en", split='train', streaming=True).shuffle(buffer_size=10_000)
ds = ds.batch(batch_size=128)

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

In [10]:
import re
import evaluate
import pandas as pd
metric = evaluate.load("Abdulmohsena/classicier")
pool = pd.DataFrame(columns=['ar', 'en'])

def process_and_filter(batch):
    df = pd.DataFrame(batch['translation'])
    df = df[~df['ar'].str.contains(r"[A-Za-z1-9\(\)]")]
    df = df[~df['en'].str.contains(r"[1-9\(\)]")]
    df = df[df['ar'].str.len() > 30]
    df.reset_index(inplace=True, drop=True)
    scores = metric.compute(texts=df.loc[:, 'ar'].tolist())
    df['classicality'] = scores['classical_score']
    
    return df[scores['classical_score'] > 0.1]

In [11]:
for batch in ds:
    df = process_and_filter(batch)
    pool = pd.concat([pool, df], ignore_index=True)
    if len(pool) > 100:
        break

In [12]:
pool

,ar,en,classicality
0,ولم يُدرج العراق في القائمة نظراً إلى فراغ الس...,Iraq was not included in the list because of t...,0.216192
1,وكلّ هذه الظروف تقوّض بشدّة مبدأ افتراض البراء...,"In these conditions, the presumption of innoce...",0.160110
2,وقد كان عليها أن تصر في طلبها إلى أن سُمح لها ...,They had to insist to obtain visiting rights.,0.817485
3,فحيازة مثل تلك المواد تحدِث طلباً عليها.,The possession of child pornography creates de...,0.903683
4,واشتد عليه مرض السل وهو في الحبس.,His tuberculosis became worse while in custody.,0.564334
...,...,...,...
96,ويَعْتَبِر النيل من حياة الآخرين جرماً بالغ ال...,Any infringement of this right is a very serio...,0.944233
97,والفجوة الفاصلة بين الشمال والجنوب وبين الأثري...,The gap between North and South and between ri...,0.224383
98,ألف- حالة أعمال أفرقة التشاور غير الرسمية,A. Status of work of the informal consultative...,0.207799
99,والأمية بين الكبار منعدمة تقريباً.,Adult illiteracy is virtually non-existent.,0.187589


In [118]:
pool.sample().values

array([[':: الوسم والتتبع؛', 'marking and tracing;', 0.594916582107544]],
      dtype=object)